In [1]:
import numpy as np
import pickle as pickle
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

In [105]:
def visualize_process(final_history, objective_labels=["L_RSR", "L_GPR"], interval=500):
    num_generations = len(final_history)

    # Convert to numpy arrays if not already
    final_history = [np.array(gen) for gen in final_history]

    # Determine bounds across all generations
    min_obj1 = min(np.min(gen[:, 0]) for gen in final_history)
    max_obj1 = max(np.max(gen[:, 0]) for gen in final_history)
    min_obj2 = min(np.min(gen[:, 1]) for gen in final_history)
    max_obj2 = max(np.max(gen[:, 1]) for gen in final_history)

    padding_obj1 = (max_obj1 - min_obj1) * 0.1
    padding_obj2 = (max_obj2 - min_obj2) * 0.1
    xlim = (min_obj1 - padding_obj1, max_obj1 + padding_obj1)
    ylim = (min_obj2 - padding_obj2, max_obj2 + padding_obj2)

    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter([], [], c='red')
    ax.set_xlabel(objective_labels[0])
    ax.set_ylabel(objective_labels[1])
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.grid(True)

    def update(frame):
        data = final_history[frame]
        scatter.set_offsets(data)
        ax.set_title(f"Pareto Front - Generation {frame + 1}")
        return scatter,

    ani = animation.FuncAnimation(fig, update, frames=num_generations, interval=interval, blit=True)
    plt.close(fig)
    return ani


In [106]:
def dominate(a, b):
    if a[0] < b[0] and a[1] < b[1]:
        return 1
    elif b[0] < a[0] and b[1] < a[1]:
        return -1
    else:
        return 0

def arkiv_proccess(history):
    arkiv = history[0][:]
    final_history = [arkiv[:]]  

    for i in range(1, len(history)):
        current_gen = history[i]
        valid_new_idx = []
        remove_arkv_idx = []

        for j in range(len(current_gen)):
            is_valid = True
            for k in range(len(arkiv)):
                check = dominate(arkiv[k], current_gen[j])
                if check == 1:
                    is_valid = False
                    break
                elif check == -1:
                    remove_arkv_idx.append(k)
            if is_valid:
                valid_new_idx.append(j)

        remove_arkv_idx = list(set(remove_arkv_idx))
        arkiv = [ind for idx, ind in enumerate(arkiv) if idx not in remove_arkv_idx]

        for j in valid_new_idx:
            arkiv.append(current_gen[j])

        final_history.append(arkiv[:])  
    return final_history


# Visualize pareto font

In [116]:
path = r"D:\EnhanceGARAG\nsgaii_logs\ngsgaii_golden_answer_0.2_0.pkl"
with open(path, 'rb') as f:
    history = pickle.load(f)
final_history = arkiv_proccess(history)
ani = visualize_process(final_history)
HTML(ani.to_jshtml())